# NO MACHIING LEARNING LIBRARY USED FOR TRAINING/PREDICTION/CLASSIFCATION

In [95]:
from nltk.stem import PorterStemmer
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.model_selection import train_test_split

# Why NLP?
combine manylabs with training('train.csv') daset
apply count vectorizer to both of them together
only train a model based on logistic regression using training dataset
use trained model to analyze sentiment of manylabs highpower and low power column
create a CSV FILE TO store sentiment for [highpower] [lowpoer] for each candidate

In [77]:
class Logistic:
    def __init__(self,rate,iteration):
        self.rate = rate
        self.weights = None
        self.iteration = iteration
    
    def sigmoid(self,z):
        return 1/(1+np.exp(-z))
    
    def loss(self,h, y):
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    def gradient_descent(self,X, h, y):
        return np.dot(X.T, (h - y)) / y.shape[0]

    def update_weight_loss(self,weight, learning_rate, gradient):
        return weight - learning_rate * gradient
    
    def initialize_parameters(self,X):
        self.weights = np.random.randn(len(X[0])) * 0.01
        
    def fit(self,X,y):
        self.initialize_parameters(X)
        #gradient descent
        for i in range(self.iteration):
            h = self.sigmoid(np.dot(X,self.weights))
            gradient = self.gradient_descent(X,h,y)
            self.weights = self.update_weight_loss(self.weights,self.rate,gradient)
            if i % 10 == 0:
                print('current iter is: ', i)
    def predict(self,X):
        ans = self.sigmoid(np.dot(X,self.weights))
        for i in range(len(ans)):
            if ans[i] < 0.5:
                ans[i] = 0
            else:
                ans[i] = 1
        return ans
    
    def test_accuracy(self,X,y):
        cor = 0
        total = len(y)
        for i in range(total):
            if X[i] == y[i]:
                cor+=1
        
        print("predict accuracy is: ", cor/total)
        return cor/total
        

In [93]:
def create_array_for_training_testing_sent_data():
    train = pd.read_csv('data/train.csv',encoding='latin-1')
    test = pd.read_csv('data/test.csv',encoding='latin-1')
    train = train[0:10000]
#     test = test[0:8000]
    #create list to hold all data
    train_tags = train['Sentiment'].values
    train_text = train['SentimentText'].values

    return train_text,train_tags

In [94]:
train_text,train_tags = create_array_for_training_testing_sent_data()

# CREATE df to store manylab text column

In [91]:
type(train_test)

numpy.ndarray

# Clean dataset into regular expression

In [107]:
def preprocess_reviews(reviews):

    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    NO_SPACE = ""
    SPACE = " "
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(train_text)
# reviews_test_clean = preprocess_reviews(reviews_test)

# n-grams size = 1,2
# STOP_WORDS

In [108]:
def data_preprocessing(reviews_train_clean):
    stop_words = ['in', 'of', 'at', 'a', 'the']
    ngram_vectorizer = CountVectorizer(binary=True,ngram_range=(1,2),stop_words= stop_words)
    ngram_vectorizer.fit(reviews_train_clean)
    X = ngram_vectorizer.transform(reviews_train_clean)
    return X
    

In [142]:
X = data_preprocessing(reviews_train_clean)

In [143]:
X = ngram_vectorizer.transform(reviews_train_clean)

In [132]:
x_train,x_val,y_train,y_val = train_test_split(X,train_tags,train_size = 0.75)

/usr/lib/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [148]:
x_val

<2500x20075 sparse matrix of type '<class 'numpy.int64'>'
	with 30092 stored elements in Compressed Sparse Row format>

In [140]:
def convert_sparse_to_np_array(x_train,y_train):
    return csr_matrix(x_train).toarray(),csr_matrix(y_train).toarray()  

In [146]:
x_train,y_train = convert_sparse_to_np_array(x_train,y_train)  #convert a scipy.sparse.csr.csr_matrix to np array

In [138]:
log_reg = Logistic(0.5,100)

In [144]:
x_train.shape

(7500, 20075)

In [145]:
y_train.shape

(1, 7500)

In [139]:
log_reg.fit(x_train,y_train)

ValueError: shapes (20075,7500) and (1,7500) not aligned: 7500 (dim 1) != 1 (dim 0)

In [80]:
a_fial = log_reg.predict(a)

In [81]:
log_reg.test_accuracy(a_fial,train_tags)

predict accuracy is:  0.7985


0.7985